Assume you have a quadrat with four sides (or N sides), where each side has three letters (imagine they are points on the side of the quadrat).

Your task is the find the least amount of words of at least len = 3 which touches each letter at least once. When picking the next letter, said letter must be on a different side than the current letter. The first letter of the next word must be the same as the last letter of the previous word (i.e. said letter belongs to two words at once).

In [1]:
import heapq
from collections import defaultdict
from tqdm.notebook import tqdm

In [2]:
CUTOFF = 4  # We can assume that a good solution requires four words or less. So if we have to use more than four words, it is not a candidate.

In [3]:
sides = {
    1: ['w', 'f', 'i'],
    2: ['n', 'c', 'y'],
    3: ['g', 't', 'a'],
    4: ['h', 'e', 'm'],
}

# Let's transform those into a tuple which contains letter and side:
def generate_nodes(sides):
    return {(k, letter) for k in range(1, 5) for letter in sides[k]}
nodes = generate_nodes(sides)

# Depending on which side we are on, we need to decide which possible letters are next:
def generate_possible_nexts(nodes):
    possible_nexts = {}
    for current_node in nodes:
        possible_nexts[current_node] = []
        for node in nodes:
            if node[0] != current_node[0]:
                possible_nexts[current_node].append(node)
    return possible_nexts
possible_nexts = generate_possible_nexts(nodes)

In [4]:
def is_possible(current_position, letter):
    return letter in [c for side, c in possible_nexts[current_position]]

In [5]:
def get_all_possible(current_position, letter):
    return [node for node in possible_nexts[current_position] if letter == node[1]]

In [6]:
# Read in word list:
with open('words.txt', 'rt') as f:
    words = [word.strip() for word in f.readlines()]

In [7]:
def generate_paths(input_list):
    # Start with a list containing an empty path
    paths = [[]]
    
    # Iterate through each step (inner list) in the journey
    for step in input_list:
        # For each existing path, create new paths by appending each option in the current step
        paths = [path + [option] for path in paths for option in step]
    
    return paths

# Let's have a sample path to test with:
sample_path = [[(4, 'h')], [(1, 'i'), (4, 'i')], [(3, 't')], [(4, 'e')]]
generate_paths(sample_path)

[[(4, 'h'), (1, 'i'), (3, 't'), (4, 'e')],
 [(4, 'h'), (4, 'i'), (3, 't'), (4, 'e')]]

In [8]:
def is_valid_path(path):
    side = -1
    for step in path:
        if step[0] == side:
            return False
        else:
            side = step[0]
            
    return True

assert is_valid_path(generate_paths(sample_path)[0]) is True
assert is_valid_path(generate_paths(sample_path)[1]) is False

In [9]:
# The conditions are: When you trace a word through the sides, no two subsequent side values might be the same.
# So we have to get all possible side indices for a given word and then generate all possible paths.
def get_all_paths(word, current_position=None):
    if current_position:
        visitable_nodes = [[node for node in nodes if node[1] == letter] for letter in word[1:]]
        return [path for path in generate_paths(visitable_nodes) if is_valid_path(path)]

    else:
        visitable_nodes = [[node for node in nodes if node[1] == letter] for letter in word]
        return [path for path in generate_paths(visitable_nodes) if is_valid_path(path)]        


get_all_paths('white', (1, 'w'))

[[(4, 'h'), (1, 'i'), (3, 't'), (4, 'e')]]

In [10]:
get_all_paths('white')

[[(1, 'w'), (4, 'h'), (1, 'i'), (3, 't'), (4, 'e')]]

In [11]:
# Filter out impossible words: We will loop through all words and distill a ton of candidates which are compatible with our quadrant.
possible_words = []
for word in tqdm(words):
    paths = get_all_paths(word)
    if len(paths) > 0:
        possible_words.append(word)

possible_words = sorted(possible_words, key=lambda x: -len(x))

  0%|          | 0/198422 [00:00<?, ?it/s]

In [12]:
print(f'After pruning, there are {len(possible_words)} words left to consider, from originally {len(words)}.')

After pruning, there are 778 words left to consider, from originally 198422.


In [13]:
%%time
# Solution algorithm: For each word to start with, we will check the set of letters left to hit 
# and then loop through all other words and create the new set of letters left.
# We will check whether we reached a solution and then break off, but otherwise we will keep going.
# Each proposed solution will be stored in a tuple with (score, [words used], {letters left}, current_position),
# where the score is calculated from 100 * len(current_positions) + len([words_used]).
# We will use heapq to manage the queue and we will prune all solutions which exceed the allowed max length of words.

def score_candidate(history, left):
    return 100*len(left) + len(history)

    
# We are interested in only one solution:
solution = None
solution_dict = defaultdict(list)
len_longest_solution = CUTOFF + 1
k = 0

# Initial candidate generation:
candidates = []
for word in possible_words:
    paths = get_all_paths(word)

    for path in paths:
        left = nodes - set(path)
        history = [word]
        score = score_candidate(history, left)

        if left:
            heapq.heappush(candidates, (score, history, left, path[-1]))
        else:
            print(f'Holy cow we already have a winner: {word}')
            solution = word
            len_longest_solution = 1


while True:
    if candidates:
        score, history, left, cur_pos = heapq.heappop(candidates)
    else:
        break

    # Loop through all words which start with the same letter as the current one ended:
    for word in [w for w in words if w[0] == cur_pos[1]]:
        paths = get_all_paths(word)

        for path in paths:
            # Counts the steps for funs an giggles:
            if k % 10_000 == 9_999:
                print(f'- Reached step {k+1}')
            k += 1
            
            new_left = left - set(path)
            new_history = history[:] + [word]
            new_score = score_candidate(new_history, new_left)

            if not new_left:
                if len(new_history) <= len_longest_solution:
                    solution = new_history
                    len_longest_solution = len(solution)
                    print(f'Found new potentially shortest solution of length {len_longest_solution} in step {k} with {len(candidates)} remaining.')
                    solution_dict[len(solution)].append(solution)
   
            if new_score < score and len(new_history) < len_longest_solution - 1:
                heapq.heappush(candidates, (new_score, new_history, new_left, path[-1]))

Found new potentially shortest solution of length 4 in step 85 with 797 remaining.
Found new potentially shortest solution of length 4 in step 86 with 797 remaining.
Found new potentially shortest solution of length 4 in step 110 with 797 remaining.
Found new potentially shortest solution of length 4 in step 140 with 797 remaining.
Found new potentially shortest solution of length 4 in step 179 with 796 remaining.
Found new potentially shortest solution of length 4 in step 180 with 796 remaining.
Found new potentially shortest solution of length 4 in step 181 with 796 remaining.
Found new potentially shortest solution of length 4 in step 202 with 796 remaining.
Found new potentially shortest solution of length 4 in step 203 with 796 remaining.
Found new potentially shortest solution of length 4 in step 207 with 795 remaining.
Found new potentially shortest solution of length 4 in step 208 with 795 remaining.
Found new potentially shortest solution of length 4 in step 232 with 795 remai

In [14]:
print(solution)

['fetch', 'highwayman']


In [15]:
# What is the optimal solution

In [16]:
solution_dict[min(solution_dict.keys())]

[['fetich', 'highwayman'], ['fetch', 'highwayman']]